In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

device = "cuda:2"  # the device to load the model onto

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "/mnt/sda/agent_mxz/models/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/mnt/sda/agent_mxz/models/Qwen2-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def get_response(prompt):
    # Define the prompt and messages
    if not prompt:
        prompt = "你是一个数据科学家，这里有一个表结构，表ioms_alarm_current 字段：alarm_id, line, system, device_id, level, station, sub_frame, slot, port, alias, code, content, target_id, abnormal_type, trigger_reason, count, alarm_time_first, alarm_time_last, confirm_time, status, source, alarm_type, remark, 查询一下最近三天的告警信息。要求你仅输出SQL代码,不加入任何非代码以外的解释或者背景信息，不要解释，不要解释，不要解释"

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Apply the chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Define the generation configuration with adjusted temperature
    generation_config = GenerationConfig(
        temperature=0.001,  # Adjust the temperature as needed
        max_new_tokens=200,
        do_sample=False
    )

    # Generate the response
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            generation_config=generation_config
        )

    # Extract the generated tokens
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode the response
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

In [18]:
prompt = """你是一个数据科学家，这里有一个表结构，
下面是建表语句：CREATE TABLE IF NOT EXISTS `ioms_alarm_history` (
  `alarm_id` varchar(20) NOT NULL COMMENT '告警id',
  `line_name` varchar(50) NOT NULL COMMENT '线路名称',
  `station_name` varchar(50) NOT NULL COMMENT '站点名称',
  `subsystem_name` varchar(50) NOT NULL COMMENT '子系统名称',
  `device_name` varchar(100) NOT NULL COMMENT '设备名称',
  `content` varchar(100) NOT NULL COMMENT '告警内容',
  `trigger_reason` varchar(128) DEFAULT NULL COMMENT '触发原因',
  `device_id` varchar(20) NOT NULL COMMENT '设备id',
  `alarm_code` varchar(52) NOT NULL COMMENT '告警码',
  `alarm_level_id` varchar(20) NOT NULL COMMENT '告警等级id',
  `alarm_level_name` varchar(20) NOT NULL COMMENT '告警等级名称',
  `target_id` varchar(20) DEFAULT NULL COMMENT '指标id',
  `target_title` varchar(100) DEFAULT NULL COMMENT '指标标题',
  `target_name` varchar(50) DEFAULT NULL COMMENT '指标名称',
  `abnormal_type` int(11) DEFAULT NULL COMMENT '异常类型，0-预警规则，1-智能检测，2-同环比，3-陡增，4-陡降',
  `count` int(11) NOT NULL COMMENT '告警次数',
  `device_type_name` varchar(50) NOT NULL COMMENT '设备类型名称',
  `sub_frame` varchar(100) NOT NULL DEFAULT '' COMMENT '子架、机框、模块',
  `slot` varchar(100) NOT NULL DEFAULT '' COMMENT '槽位、板卡、子模块',
  `port` varchar(100) NOT NULL DEFAULT '' COMMENT '端口',
  `alias` varchar(100) NOT NULL DEFAULT '' COMMENT '单元名称',
  `handler` varchar(50) NOT NULL COMMENT '处理人名称',
  `clear_time` timestamp NULL DEFAULT NULL COMMENT '清除时间',
  `alarm_time_first` timestamp NULL DEFAULT NULL COMMENT '第一次告警时间',
  `alarm_time_last` timestamp NULL DEFAULT NULL COMMENT '最近一次告警时间',
  `confirm_time` timestamp NULL DEFAULT NULL COMMENT '告警确认时间',
  `time_interval` bigint(20) DEFAULT NULL COMMENT '告警持续时间(单位:s)',
  `status` varchar(50) NOT NULL COMMENT '状态',
  `alarm_type` tinyint(4) NOT NULL DEFAULT '1' COMMENT '告警类型: 1.告警, 2.预警',
  `cause` varchar(1024) DEFAULT NULL COMMENT '告警原因',
  `remark` varchar(100) DEFAULT NULL COMMENT '备注',
  PRIMARY KEY (`alarm_id`),
  KEY `ind_alarm_history_line_system_level` (`line_name`,`subsystem_name`,`alarm_level_id`,`station_name`,`alarm_time_last`),
  KEY `ind_alarm_history_line_station_level` (`line_name`,`station_name`,`alarm_level_id`,`alarm_time_last`),
  KEY `ind_alarm_history_line_level` (`line_name`,`alarm_level_id`,`alarm_time_last`),
  KEY `ind_alarm_history_line_device_level` (`line_name`,`device_id`,`device_name`,`station_name`,`alarm_time_last`),
  KEY `ind_alarm_history_alarm_code` (`line_name`,`alarm_code`,`content`,`station_name`,`alarm_time_last`),
  KEY `ind_alarm_history_last_alarm_time` (`alarm_time_last`),
  KEY `ind_alarm_history_clear_time` (`clear_time`),
  KEY `ind_alarm_history_time_interval` (`time_interval`),
  KEY `ind_alarm_history_system` (`subsystem_name`),
  KEY `ind_alarm_history_device` (`device_id`),
  KEY `ind_alarm_history_device_type_name` (`line_name`,`station_name`,`subsystem_name`,`alarm_time_last`,`device_type_name`),
  KEY `ind_alarm_history_line_time_interval` (`line_name`,`station_name`,`subsystem_name`,`time_interval`,`alarm_time_last`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='历史告警表';
表ioms_alarm_history 字段：alarm_id, line, system, device_id, level, station, sub_frame, slot, port, alias, code, content, target_id, abnormal_type, trigger_reason, count, alarm_time_first, alarm_time_last, confirm_time, status, source, alarm_type, remark, 统计一下哪些模块最容易发生故障。你仅输出SQL代码,不加入任何非代码以外的解释或者背景信息，不要解释，不要解释，不要解释"""
get_response(prompt)

```sql
SELECT 
    sub_frame, 
    COUNT(*) as fault_count
FROM 
    ioms_alarm_history
WHERE 
    status = '故障'
GROUP BY 
    sub_frame
ORDER BY 
    fault_count DESC
LIMIT 10;
```
